In [174]:
import bs4
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import time
import random
import statsapi
from IPython.display import clear_output, display

REQUESTS_PER_MINUTE = 20
wpa_df = pd.DataFrame

In [175]:
games = statsapi.schedule(start_date='2023-01-01', end_date='2023-12-31')

In [176]:
team_mapping = {
    "Arizona Diamondbacks":"ARN",
    "Atlanta Braves": "ATL",
    "Baltimore Orioles": "BAL",
    "Boston Red Sox": "BOS",
    "Chicago White Sox": "CHA",
    "Chicago Cubs": "CHN",
    "Cincinnati Reds": "CIN",
    "Cleveland Guardians": "CLE",
    "Colorado Rockies": "COL",
    "Detroit Tigers": "DET",
    "Houston Astros": "HOU",
    "Kansas City Royals": "KCA",
    "Los Angeles Angels": "LAA",
    "Los Angeles Dodgers": "LAN",
    "Miami Marlins": "MIA",
    "Milwaukee Brewers": "MIL",
    "Minnesota Twins": "MIN",
    "New York Yankees":"NYA",
    "New York Mets": "NYN",
    "Oakland Athletics": "OAK",
    "Philadelphia Phillies": "PHI",
    "Pittsburgh Pirates": "PIT",
    "San Diego Padres": "SDN",
    "San Francisco Giants": "SFN",
    "Seattle Mariners": "SEA",
    "St. Louis Cardinals": "STL",
    "Tampa Bay Rays": "TBA",
    "Texas Rangers": "TEX",
    "Toronto Blue Jays": "TOR",
    "Washington Nationals": "WAS",
}

In [180]:
df = pd.DataFrame.from_dict(games)
reg_season_games = df[df.game_type != "E"]

In [178]:
def getBRefGameString(game) -> string:
    """given a game series, return a string that would navigate to that game on baseball reference """
    gameSeries = game.iloc[0]
    gamestring = team_mapping[game['home_name']]
    gamestring += game['game_date'].replace('-','')

    if game['game_num'] == 1:
        gamestring += '0'
    if game['game_num'] == 2:
        gamestring += '1'

    return gamestring

game_string_list = list(df[df.game_type == 'R'].apply(getBRefGameStrings,axis=1))
game_ids = list(df[df.game_type == 'R'].game_id)

NameError: name 'string' is not defined

In [135]:
def scrape(df, gameID):
    gameSeries = (df[df["game_id"] == gameID]).iloc[0]
    gs = getBRefGameString(gameSeries)
    url = f"https://www.baseball-reference.com/boxes/{gs[0:3]}/{gs}.shtml"
    print(url)
    time.sleep(60/REQUESTS_PER_MINUTE)
    req = requests.get(url)
    resp = req.status_code
    d = soup(req.text, 'html.parser')
    if resp == 200:
        print('hi')
        block = [i for i in d.find_all(string=lambda text: isinstance(text, bs4.Comment)) if 'id="div_play_by_play"' in i][0]
        b = soup(str(block), 'html.parser')
        table = b.find_all('table')
        path = f"./wpa_data/{gameID}.csv"
        df = pd.read_html(str(table))[0]
        df.to_csv(path)
        print(f"Created {path}")
    else:
        print(f"Game ID: {gameID}")
        print(f"Game String: {gs}")
        return None

In [152]:
total_queries = len(game_string_list)
count = 0
for g in game_ids:
    scrape(df, g)                
    clear_output(wait=True)
    display(f"{count+1}/{total_queries} ({(count/total_queries)*100:.2f}%) queries complete, just requested {g}")
    count += 1
    if data is not None:
        full_data.append(data)

'5/2476 (0.16%) queries complete, just requested 718776'

https://www.baseball-reference.com/boxes/TEX/TEX202303300.shtml


KeyboardInterrupt: 

In [166]:
game_ids.sort()
game_ids[0]
len(game_ids)

2476

In [183]:
reg_season_games[reg_season_games.game_type == 'R'].sort(reg_season_games.game_ids)

AttributeError: 'DataFrame' object has no attribute 'sort'

In [218]:
df[(df["game_type"] == "R") & (df["status"] != "Postponed") & (df["status"] != "Cancelled")].away_name.nunique()

30

In [229]:
val_counts = df[df.game_type == "R"].game_id.value_counts()

In [239]:
df[df.game_id.isin(val_counts[val_counts.values == 2].index)].to_csv("dupes.csv")

In [243]:
df[(df.game_type == "R") & (df.status == "Final")]

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,...,venue_name,national_broadcasts,series_status,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,losing_Team
491,718780,2023-03-30T17:05:00Z,2023-03-30,R,Final,Atlanta Braves,Washington Nationals,144,120,N,...,Nationals Park,[MLBN],ATL leads 1-0,Atlanta Braves,Washington Nationals,Lucas Luetge,Patrick Corbin,None,2023-03-30 - Atlanta Braves (7) @ Washington N...,NaN
492,718781,2023-03-30T17:05:00Z,2023-03-30,R,Final,San Francisco Giants,New York Yankees,137,147,N,...,Yankee Stadium,[MLBN],NYY leads 1-0,New York Yankees,San Francisco Giants,Gerrit Cole,Logan Webb,None,2023-03-30 - San Francisco Giants (0) @ New Yo...,NaN
493,718782,2023-03-30T18:10:00Z,2023-03-30,R,Final,Baltimore Orioles,Boston Red Sox,110,111,N,...,Fenway Park,[MLB.tv Free Game],BAL leads 1-0,Baltimore Orioles,Boston Red Sox,Kyle Gibson,Corey Kluber,Félix Bautista,2023-03-30 - Baltimore Orioles (10) @ Boston R...,NaN
494,718777,2023-03-30T18:20:00Z,2023-03-30,R,Final,Milwaukee Brewers,Chicago Cubs,158,112,N,...,Wrigley Field,[],CHC leads 1-0,Chicago Cubs,Milwaukee Brewers,Marcus Stroman,Corbin Burnes,None,2023-03-30 - Milwaukee Brewers (0) @ Chicago C...,NaN
495,718776,2023-03-30T19:10:00Z,2023-03-30,R,Final,Detroit Tigers,Tampa Bay Rays,116,139,N,...,Tropicana Field,[],TB leads 1-0,Tampa Bay Rays,Detroit Tigers,Shane McClanahan,Eduardo Rodriguez,None,2023-03-30 - Detroit Tigers (0) @ Tampa Bay Ra...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,716354,2023-10-01T19:10:00Z,2023-10-01,R,Final,Washington Nationals,Atlanta Braves,120,144,N,...,Truist Park,[MLB.tv Free Game],WSH wins 2-1,Washington Nationals,Atlanta Braves,Hunter Harvey,Michael Tonkin,Kyle Finnegan,2023-10-01 - Washington Nationals (10) @ Atlan...,NaN
2963,716356,2023-10-01T19:10:00Z,2023-10-01,R,Final,San Diego Padres,Chicago White Sox,135,145,N,...,Guaranteed Rate Field,[MLB.tv Free Game],SD wins 3-0,San Diego Padres,Chicago White Sox,Rich Hill,Declan Cronin,None,2023-10-01 - San Diego Padres (2) @ Chicago Wh...,NaN
2964,716352,2023-10-01T19:10:00Z,2023-10-01,R,Final,New York Yankees,Kansas City Royals,147,118,N,...,Kauffman Stadium,[MLB.tv Free Game],KC wins 2-1,Kansas City Royals,New York Yankees,Zack Greinke,Michael King,James McArthur,2023-10-01 - New York Yankees (2) @ Kansas Cit...,NaN
2965,716364,2023-10-01T19:10:00Z,2023-10-01,R,Final,Chicago Cubs,Milwaukee Brewers,112,158,N,...,American Family Field,[MLB.tv Free Game],MIL wins 2-1,Milwaukee Brewers,Chicago Cubs,Adrian Houser,Drew Smyly,None,2023-10-01 - Chicago Cubs (0) @ Milwaukee Brew...,NaN


In [242]:
92/2+(162*15)

2476.0